In [ ]:
#The main portion of this code was taken from the below license. In addition to their detailed instructions, modifications
#were made to make it work with our data sets and to modify some of their algorithm.

#Note, this does run on docker as the data sets are smaller

In [2]:
#MIT License

#Copyright (c) 2017 Ankit Sachan

#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all
#copies or substantial portions of the Software.

In [1]:
your_name = 'perry_ronan'

!pip install mahotas
!pip install ndparse
!pip install tensorflow
%matplotlib inline

    100% |████████████████████████████████| 1.5MB 456kB/s eta 0:00:01    91% |█████████████████████████████▏  | 1.4MB 35.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for mahotas ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/87/24/2ffb31d56c7d1f21175c006697cb7e305a43fe0be0891cfdc8
Successfully built mahotas
  Running setup.py bdist_wheel for ndparse ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/1f/3b/63/8d99427fa5243ad5490f37c5dfbb78fc3153e07faafa12cd5e
Successfully built ndparse
    100% |████████████████████████████████| 41.2MB 12kB/s eta 0:00:011  1% |▌                               | 593kB 7.4MB/s eta 0:00:06    2% |█                               | 1.2MB 8.5MB/s eta 0:00:05    6% |██                              | 2.7MB 14.6MB/s eta 0:00:03    7% |██▌                             | 3.3MB 6.2MB/s eta 0:00:07    12% |████                            | 5.3MB 17.7MB/s eta 0:00:03    18% |█████▉                          | 7.5MB 12.8MB/s eta 0:00:03

In [8]:
!pip install networkx==1.9 #networkx broke backwards compatibility with these graph files
import numpy as np
import networkx as nx
import scipy as sp
import matplotlib.pyplot as plt
import os
import csv

import tensorflow as tf

from skimage.feature import daisy
from skimage import data

from collections import OrderedDict

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# Initializing dataset names
dataset_names = 'SWU4'

basepath = 'data'

# change which atlas you use, here!

atlas = 'desikan' # 'desikan' # or 'CPAC200', or 'Talairach'
dir_names = basepath + '/' + dataset_names + '/' + atlas
#basepath = "/"
#dir_names = basepath
print(dir_names)
fs = OrderedDict()
fs[dataset_names] = [root + "/" + fl for root, dirs, files in os.walk(dir_names)
                     for fl in files if fl.endswith(".gpickle")]

ps = "data/SWU4/SWU4.csv"

print("Datasets: " + ", ".join([fkey + " (" + str(len(fs[fkey])) + ")"
                                for fkey in fs]))
print("Total Subjects: %d" % (sum([len(fs[key]) for key in fs])))

data/SWU4/desikan
Datasets: SWU4 (454)
Total Subjects: 454


In [4]:
def loadGraphs(filenames, verb=False):
    """
    Given a list of files, returns a dictionary of graphs

    Required parameters:
        filenames:
            - List of filenames for graphs
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Initializes empty dictionary
    gstruct = OrderedDict()
    for idx, files in enumerate(filenames):
        if verb:
            print("Loading: " + files)
        #  Adds graphs to dictionary with key being filename
        fname = os.path.basename(files)
        gstruct[fname] = nx.read_gpickle(files)
    return gstruct

def constructGraphDict(names, fs, verb=False):
    """
    Given a set of files and a directory to put things, loads graphs.

    Required parameters:
        names:
            - List of names of the datasets
        fs:
            - Dictionary of lists of files in each dataset
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Loads graphs into memory for all datasets
    graphs = OrderedDict()
    if verb:
        print("Loading Dataset: " + names)
    # The key for the dictionary of graphs is the dataset name
    graphs[names] = loadGraphs(fs[names], verb=verb)
    return graphs

In [5]:
graphs = constructGraphDict(dataset_names, fs, verb=False)

In [6]:
import csv
# This gets age and sex, respecitvely.
tmp = csv.reader(open(ps,newline='')) # this is the whole phenotype file
pheno = OrderedDict()
triple = [[t[0].strip(), t[2], int(t[3] == '2')] for t in tmp
          if t[3] != '#' and t[2] != '#'][1:]  # female=1->0, male=2->1

for idx, trip in enumerate(triple):
    pheno[trip[0]] = trip[1:]

In [132]:
tf.reset_default_graph()


In [133]:
k = sorted(list(graphs['SWU4'].keys()))
k_id = list(key[6:11] for key in k)
k_id = k_id[0::2]
k_g1 = k[0::2]

g1 = []
for xx in k_g1:
    g1.append(graphs['SWU4'][xx])

#Create vectors of labels
age = list()
sex = list()

for key in k_id:
    sex.append(pheno[key][1])
    age.append(pheno[key][0])

In [134]:
# Load data

sex_classes = [[1,0] if i == 0 else [0,1] for i in sex]
traindata = [nx.to_numpy_matrix(i, nodelist=sorted(i.nodes())) for i in g1]
traindata = np.array([np.array(i) for i in traindata])

imtrain = traindata[0:100]
ytrain = sex_classes[0:100]

imvalid = traindata[100:200]
yvalid = sex_classes[100:200]

In [135]:
imtrain = imtrain.reshape((np.shape(imtrain)[0],70,70,1))
imvalid = imvalid.reshape((np.shape(imvalid)[0],70,70,1))

In [136]:
#Tensor Flow classification

batch_size = 32

#Prepare input data
classes = ['female','male']
num_classes = len(classes)

# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = 70

num_channels = 1

In [137]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')

## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [138]:
##Network graph params
filter_size_conv1 = 3
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 128

In [139]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [140]:
def create_convolutional_layer(input,
                num_input_channels,           
                conv_filter_size,        
                num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer

In [141]:
def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer

In [142]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [143]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

In [144]:
y_pred = tf.nn.softmax(layer_fc2,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 


In [145]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

In [146]:
total_iterations = 0
saver = tf.train.Saver()
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):

        indices_train = np.random.choice(int(0.8*imtrain.shape[0]), batch_size)
        x_batch = imtrain[indices_train]
        y_true_batch = [ytrain[i] for i in indices_train]      
        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        
        offset = int(0.8*imtrain.shape[0])
        indices_valid = np.random.choice(int(0.2*imtrain.shape[0]), batch_size)
        x_valid_batch = imtrain[indices_valid + offset]
        y_valid_batch = [ytrain[i+offset] for i in indices_valid]
        
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(imtrain.shape[0]/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(imtrain.shape[0]/batch_size)) 
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, './classify-model') 


    total_iterations += num_iteration


In [147]:
session.run(tf.global_variables_initializer()) 

train(num_iteration=200)

Training Epoch 1 --- Training Accuracy:  43.8%, Validation Accuracy:  43.8%,  Validation Loss: 86.848
Training Epoch 2 --- Training Accuracy:  50.0%, Validation Accuracy:  68.8%,  Validation Loss: 110.527
Training Epoch 3 --- Training Accuracy:  53.1%, Validation Accuracy:  21.9%,  Validation Loss: 150.913
Training Epoch 4 --- Training Accuracy:  65.6%, Validation Accuracy:  31.2%,  Validation Loss: 41.260
Training Epoch 5 --- Training Accuracy:  53.1%, Validation Accuracy:  62.5%,  Validation Loss: 44.631
Training Epoch 6 --- Training Accuracy:  62.5%, Validation Accuracy:  81.2%,  Validation Loss: 21.651
Training Epoch 7 --- Training Accuracy:  75.0%, Validation Accuracy:  75.0%,  Validation Loss: 19.409
Training Epoch 8 --- Training Accuracy:  68.8%, Validation Accuracy:  78.1%,  Validation Loss: 17.822
Training Epoch 9 --- Training Accuracy:  75.0%, Validation Accuracy:  65.6%,  Validation Loss: 30.197
Training Epoch 10 --- Training Accuracy:  68.8%, Validation Accuracy:  59.4%,  V

In [148]:
#Begin Prediciton Portion.

#Restore prior saved trained model
sess = tf.Session()
saver = tf.train.import_meta_graph('classify-model.meta')

#Load the weights
saver.restore(sess,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./classify-model


In [149]:
#Access learned graph
graph = tf.get_default_graph()

#In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("y_pred:0")

#Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((np.shape(imvalid)[0], 2))

In [150]:
#Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: imvalid, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format [probabiliy_of_rose probability_of_sunflower]
#print(result)

In [151]:
accuracy = []
for ii in range(len(result)): #performance will change over time
    acc = ((result[ii,0] > 0.5) == yvalid[ii][0])
    #print(acc)
    accval = (float(np.sum(acc))/float(len(result)))
    accuracy.append(accval)
    #print('Accuracy:',accval)

print('Overall Accuracy:',str(np.sum(accuracy)))

f1_out = 0
thresh = 0
for i in np.arange(0.0, 1, 0.05):
    f1_test = 0
    for ii in range(len(result)):
        f1_test += ((result[ii,0] > i) == yvalid[ii][0])
    f1_test = f1_test / float(len(result))
    if f1_test > f1_out:
        f1_out = f1_test
        thresh = i

print(('Best Accuracy: %s at Threshold %s') % (str(f1_out),str(thresh)))

Overall Accuracy: 0.69
Best Accuracy: 0.69 at Threshold 0.15
